# Search all workspaces for content

This small example Python notebook:

1. Query the Bluescape API to get a list of all workspace IDs the user has access to
2. Search each retrieved workspace ID for the elements we're interested in

Before we can run the code, we need to register an "application"
[here](https://admin.apps.us.bluescape.com/applications/all_organizations):

It should look similar to this:

![Bluescape application registration screenshot](./bluescape-application-registration.png)

If you can't see a screen like that, you may not have permissions and will have to ask your
organization admin.

You'll also need to grab a JWT access token, which is described
[here](https://community.bluescape.com/t/api-application-authorization/2025).

I've used `https://localhost:8080` as the OAuth redirect URI, even though I don't have
a server running there (which is fine).

With both of those in hand, you'll need this URL (update with your own client ID token and redirect URL):

```txt
  https://api.apps.us.bluescape.com/authorize?response_type=token&client_id=<client_id>&redirect_uri=<redirect_url>
```

For example:
```txt
    https://api.apps.us.bluescape.com/authorize?response_type=token&client_id=1234567c-89f0-12f3-4a5b-6a7a8e9b0123&redirect_uri=https://localhost:8080
```

Then you can grab the access token from directly from the URL as described in the
[documentation](https://community.bluescape.com/t/api-application-authorization/2025).


In [ ]:
import json
import requests
from typing import Any, List, Dict, Optional

## Helper functions

In [25]:
def load_secret(file_path: str) -> str:
    """A small helper function that returns a .txt file contents as a string"""
    with open(file_path, mode="r") as file:
        return file.read().strip()


def bearer_auth(access_token: str) -> str:
    """Create the authentication bearer header value"""
    return f"Bearer {access_token}"


def create_auth_header(access_token: str) -> Dict[str, str]:
    """Create the header dictionary with the authorization bearer token"""
    return {"Authorization": bearer_auth(access_token)}

## Configuration

In [27]:
# The production US Shared API
bluescape_api = "https://api.apps.us.bluescape.com"

# Load the secret Bluescape API access token
application_client_id = "--Your-application-client-id--"
access_token = "--Your-access-token-here--or-store-it-in-a-file-outside-of-git-and-read-it-with-load_secret()"
# access_token = load_secret("c:\\private\your-access-token-file.txt")


## Get user Workspace IDs

To search across all the workspaces the user has access to,
we first need to get a list of all the workspace IDs the user can access.

We'll loop through the detailed workspace objects extracting the workspace IDs.

In [ ]:
def get_user_workspaces_ids(
    access_token: str,
) -> List[str]:
    """ Get a list of the current user's workspace IDs.
        See: https://api.apps.us.bluescape.com/docs/doc/us/isam#/User/UserWorkspacesMeController_getSessionUserWorkspaces
    """

    # Request the workspaces URL and retrieve the workspace objects for the current user
    url = f"{bluescape_api}/v3/users/me/workspaces"
    response = requests.get(url, headers=create_auth_header(access_token))
    if response.status_code == 404:
        return []
    response_json = response.json()

    # Parse the workspace IDs out of the response JSON object
    # Response objects look like this:
    # "workspaces": [
    #   {  "id": "qUXmyrHnCSS3JbgmJjxL",
    #      ....
    # For full details, see the URL above and examine the return schema.
    if "workspaces" in response_json:
        workspaces = response_json["workspaces"]
        return [ data["id"] for data in workspaces]
    raise Exception(json.dumps(response_json, indent=4))

# Test `get_user_workspaces_ids()` by using it to find all the workspace IDs we have access to
workspace_ids = get_user_workspaces_ids(access_token)
example_workspace_id = workspace_ids[0]
print(f"Found: {len(workspace_ids)} workspace IDs")


## Search workspace

A function that will search a single workspace for certain element types with the given text.

In [ ]:
def search_elements(
    access_token: str,
    workspace_id: str,
    query: Optional[str],
    element_types: Optional[List[str]] = None,
) -> List[Dict[str, Any]]:
    """ Search in the given workspace for all elements with the given query text.

        See: https://api.apps.us.bluescape.com/docs/doc/us/elementary#/Elements/SearchController.searchElements

        From the schema, `element_types` can be any of:
            Text, LegacyNote, Icon, Image, Document, Grid, Browser, Video, Stroke, Shape, Line,
            LinkedDocument, Canvas, Selection, Window, Whiteboard

        Note that this does not search inside the content of files (like inside .docx or .pdf).
    """
    # Build the search request body
    request_body_json: Dict[str,Any] = {
        "includeComments": False,
        "orderDirection": "asc",
        "query": query,
    }
    if element_types is not None:
        request_body_json["filterTypes"] = element_types

    # Query the Bluescape search API URL
    url = f"{bluescape_api}/v3/workspaces/{workspace_id}/search"
    headers = create_auth_header(access_token)
    headers['Content-Type'] = 'application/json' # Add content type to our headers
    response = requests.post(url, headers=headers, data=json.dumps(request_body_json))
    if response.status_code == 404:
        return []
    response_json = response.json()

    # Process the response and extract the elements that match our search query
    if "data" in response_json:
        elements = response_json["data"]
        return elements
    raise Exception(json.dumps(response_json, indent=4))


## `main()`

Use our results so far to loop over all the workspaces and search each for what we want

In [ ]:
# For each workspace I have access to:
for workspace_id in workspace_ids:
    # Search for all elements in the workspace
    results = search_elements(access_token, example_workspace_id, "a")

    # Do something with these results - here I just print them out
    for result in results:
        element = result["element"]
        print(element) 
